# SQL Exercises

This exercise is about using SQL to retrieve information from a database.

We will work with a pandas dataframe and use pandas' built-in support for interacting with SQL databases via the `sqlite3` standard library module.

## Examples

We first define a couple of helper functions to open a database connection to a temporary file and to convert the output of a `sqlite3` query to a pandas dataframe to allow us to exploit the rich display of dataframes as HTML tables in Jupyter.

In [1]:
import sqlite3
import pandas as pd
import tempfile

def create_temporary_database_connection() -> sqlite3.Connection:
    temporary_file = tempfile.NamedTemporaryFile()
    return sqlite3.connect(temporary_file.name), temporary_file

def sqlite3_cursor_to_dataframe(cursor: sqlite3.Cursor) -> pd.DataFrame:
    dataframe = pd.DataFrame(
        data=cursor.fetchall(),
        columns=[c[0] for c in cursor.description],
    )
    if "index" in dataframe.columns:
        dataframe.set_index("index", inplace=True)
        dataframe.index.name = ""
    return dataframe


As an initial quick demonstration, we will create a simple dataframe with `pandas`, and then use the `DataFrame.to_sql` method to write the dataframe to a temporary database connection as a table `example_table`.

In [2]:
dataframe = pd.DataFrame({"numeric": [0, 1, 2], "text": ["A", "B", "C"]})

connection, temporary_file = create_temporary_database_connection()

dataframe.to_sql("example_table", con=connection)

3

As a first example of running an SQL query on the resulting database table, let's retrieve all values.

In [3]:
results = connection.execute("""
    SELECT *
    FROM example_table
""")

sqlite3_cursor_to_dataframe(results)

,numeric,text
,,
0,0,A
1,1,B
2,2,C


We can also retrieve particular rows that match a condition:

In [4]:
sqlite3_cursor_to_dataframe(
    connection.execute(
        """
        SELECT *
        FROM example_table
        WHERE numeric=0
        """
    )
)

,numeric,text
,,
0,0,A


and columns:

In [5]:
sqlite3_cursor_to_dataframe(
    connection.execute(
        """
        SELECT numeric
        FROM example_table
        """
    )
)

,numeric
0,0
1,1
2,2


We clean-up the database connection and associated temporary file.

In [6]:
connection.close()
temporary_file.close()

Now let's get started with the exercises. First, let's download a dataset from `scikit-learn`.

In [7]:
from sklearn.datasets import fetch_california_housing
data_california = fetch_california_housing()

Let's convert this to a dataframe so we can play with it:

In [8]:
california = pd.DataFrame(data=data_california.data, columns=data_california.feature_names)
california['target'] = data_california.target
california.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


And as before we create a temporary file and database connection and write the dataframe to it as a new table

In [9]:
connection, temporary_file = create_temporary_database_connection()

_ = california.to_sql("california", con=connection)

## Finding rows with high target variable

We can query this to find the rows with `target` value greater than 4:

In [10]:
sqlite3_cursor_to_dataframe(
    connection.execute(
        """
        SELECT *
        FROM california
        WHERE target > 4
        """
    )
)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
,,,,,,,,,
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.52600
89,1.2434,52.0,2.929412,0.917647,396.0,4.658824,37.80,-122.27,5.00001
128,7.5544,40.0,7.631498,1.030581,1616.0,2.470948,37.83,-122.21,4.11500
140,6.3624,30.0,5.615385,0.730769,126.0,2.423077,37.81,-122.18,4.83300
155,8.8793,52.0,8.972868,1.131783,861.0,3.337209,37.81,-122.23,4.10300
...,...,...,...,...,...,...,...,...,...
20428,8.7288,6.0,8.715842,1.102970,3385.0,3.351485,34.23,-118.83,4.25800
20436,12.5420,10.0,9.873315,1.102426,1179.0,3.177898,34.21,-118.69,5.00001
20438,7.2137,17.0,8.240664,1.033195,761.0,3.157676,34.26,-118.79,4.01900


We can also find how many such rows there are, using the `COUNT` function:

In [11]:
sqlite3_cursor_to_dataframe(
    connection.execute(
        """
        SELECT COUNT(*) AS TotalHighTarget
        FROM california
        WHERE target > 4
        """
    )
)

,TotalHighTarget
0,1744


To get an idea of the distribution of values in the target column, we can use some aggregate SQL functions. Compute the average, minimum and maximum value of the target variable in the table:

In [12]:
sqlite3_cursor_to_dataframe(
    connection.execute(
        """
        SELECT AVG(target) AS AverageTarget, MIN(target) AS MinTarget, MAX(target) AS MaxTarget
        FROM california
        """
    )
)

,AverageTarget,MinTarget,MaxTarget
0,2.068558,0.14999,5.00001


The results should show that the target values range between approximately 0.15 and 5, so our choice of 4 as a "high" target may be reasonable.

## Focus on older buildings

Find the rows where `HouseAge` is greater than 50 and `Population` is more than 1000.

In [13]:
sqlite3_cursor_to_dataframe(
    connection.execute(
        """
        SELECT *
        FROM california
        WHERE HouseAge > 50
        AND Population > 1000
        """
    )
)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
,,,,,,,,,
6,3.6591,52.0,4.931907,0.951362,1094.0,2.128405,37.84,-122.25,2.992
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25,2.414
9,3.6912,52.0,4.970588,0.990196,1551.0,2.172269,37.84,-122.25,2.611
11,3.2705,52.0,4.772480,1.024523,1504.0,2.049046,37.85,-122.26,2.418
12,3.0750,52.0,5.322650,1.012821,1098.0,2.346154,37.85,-122.26,2.135
...,...,...,...,...,...,...,...,...,...
18899,2.1976,52.0,4.590323,1.137097,1519.0,2.450000,38.11,-122.25,0.979
18902,2.6708,52.0,4.863894,1.207940,1238.0,2.340265,38.11,-122.26,1.097
19117,3.3011,52.0,4.616702,1.004283,1070.0,2.291221,38.23,-122.64,2.523


And count how many rows like these there are:

In [14]:
sqlite3_cursor_to_dataframe(
    connection.execute(
        """
        SELECT COUNT(*) AS OldAndBusy
        FROM california
        WHERE HouseAge > 50
        AND Population > 1000
        """
    )
)

,OldAndBusy
0,521


## More advanced keywords

Find the 5 rows with the highest number of average bedrooms, which are less than 30 years old.

**Hint:** You will need the `ORDER BY` and `LIMIT` keywords.
`ORDER BY` is followed by a column name and a sorting direction (`ASC` or `DESC` for ascending or descending, respectively).
`LIMIT` is followed by the maximum number of results we want to retrieve

In [15]:
sqlite3_cursor_to_dataframe(
    connection.execute(
        """
        SELECT *
        FROM california
        WHERE HouseAge < 30
        ORDER BY AveBedrms DESC
        LIMIT 5
        """
    )
)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
,,,,,,,,,
11862,2.6250,25.0,59.875000,15.312500,28.0,1.750000,40.27,-121.25,0.675
12447,1.6154,17.0,62.422222,14.111111,83.0,1.844444,33.97,-114.49,0.875
9676,3.2431,14.0,52.848214,11.410714,265.0,2.366071,37.64,-119.02,2.214
1240,3.1250,11.0,47.515152,11.181818,82.0,2.484848,38.42,-120.19,0.775
1913,4.0714,19.0,61.812500,11.000000,112.0,2.333333,39.01,-120.06,4.375


Again we close the connection and temporary file to clean up correctly.

In [16]:
connection.close()
temporary_file.close()